<a href="https://colab.research.google.com/github/carolineliem904-max/capstone4/blob/main/construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00


In [ ]:
import os, yaml
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change this if your zip is in a different Drive folder
ZIP_PATH = "/content/drive/MyDrive/dataset/construction_safety.zip"

In [ ]:
# Where to extract (fast local disk)
EXTRACT_DIR = "/content"
DATASET_DIR = "/content/construction_safety"

**1. unzip dataset**




In [ ]:
!rm -rf {DATASET_DIR}
!unzip -q "{ZIP_PATH}" -d "{EXTRACT_DIR}"

print("Dataset extracted. Top-level /content:")
!ls /content | head -50

print("\nDataset folder content:")
!ls "{DATASET_DIR}"

Dataset extracted. Top-level /content:
construction_safety
drive
__MACOSX
sample_data

Dataset folder content:
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


**2) FIX data.yaml (make it Ultralytics-friendly)**

In [ ]:
yaml_path = os.path.join(DATASET_DIR, "data.yaml")
assert os.path.exists(yaml_path), f"data.yaml not found at: {yaml_path}"

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

In [ ]:
# Force correct structure for Ultralytics
data["path"]  = DATASET_DIR
data["train"] = "train/images"
data["val"]   = "valid/images"
data["test"]  = "test/images"


In [ ]:
# (Optional) print classes to confirm
print("Class names in YAML:", data.get("names"))

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("\nUpdated data.yaml:")
!cat "{yaml_path}"

Class names in YAML: ['helmet', 'no-helmet', 'no-vest', 'person', 'vest']

Updated data.yaml:
train: train/images
val: valid/images
test: test/images
nc: 5
names:
- helmet
- no-helmet
- no-vest
- person
- vest
roboflow:
  workspace: personal-project-kej16
  project: construction-safety-gsnvb-jl6el
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/personal-project-kej16/construction-safety-gsnvb-jl6el/dataset/1
path: /content/construction_safety


quick check

In [ ]:
!yolo detect train model=yolov8s.pt data="{yaml_path}" imgsz=640 epochs=1 batch=16 patience=10


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/construction_safety/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=

MODEL TRAINING

In [ ]:
EPOCHS = 80
BATCH  = 16
IMGSZ  = 768

!yolo detect train model=yolov8s.pt data="{yaml_path}" imgsz={IMGSZ} epochs={EPOCHS} batch={BATCH} patience=20


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/construction_safety/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, per

SAVE OUTPUTS TO DRIVE


In [ ]:
# Find latest run folder
RUNS_DIR = "/content/runs/detect"
!ls -lt "{RUNS_DIR}" | head -20

# Usually it's /content/runs/detect/train/weights/best.pt
BEST_PT = "/content/runs/detect/train/weights/best.pt"
assert os.path.exists(BEST_PT), f"best.pt not found at: {BEST_PT}"

SAVE_DIR = "/content/drive/MyDrive/capstone4_construction_safety_768"
os.makedirs(SAVE_DIR, exist_ok=True)

!cp "{BEST_PT}" "{SAVE_DIR}/best.pt"
!cp "/content/runs/detect/train/results.png" "{SAVE_DIR}/results.png" || true
!cp "/content/runs/detect/train/confusion_matrix.png" "{SAVE_DIR}/confusion_matrix.png" || true

print("\nSaved to Drive:", SAVE_DIR)
!ls -lh "{SAVE_DIR}"

total 8
drwxr-xr-x 3 root root 4096 Feb 16 02:58 train2
drwxr-xr-x 3 root root 4096 Feb 16 02:24 train

Saved to Drive: /content/drive/MyDrive/capstone4_construction_safety_768
total 22M
-rw------- 1 root root  22M Feb 16 03:07 best.pt
-rw------- 1 root root 142K Feb 16 03:07 confusion_matrix.png
-rw------- 1 root root 165K Feb 16 03:07 results.png
